## Bag of words 

In [1]:
#reading the dataset
from datasets import load_dataset
raw_datasets = load_dataset("ag_news")

/Users/mahnaz/vscodeProjects/News-Categorization-FineTuned-BERT/venv_News-Categorization/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset ag_news (/Users/mahnaz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
100%|██████████| 2/2 [00:00<00:00, 41.21it/s]


In [7]:
#lets have a look at the data
print("keys: ",raw_datasets.keys())
print(type(raw_datasets['train']))
print(raw_datasets['train'].features)
print(f"example of train datapoint: \n {raw_datasets['train'][0]}")
print(raw_datasets['train'][0]['text'])
print(raw_datasets['train'][0]['label'])

keys:  dict_keys(['train', 'test'])
<class 'datasets.arrow_dataset.Dataset'>
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)}
example of train datapoint: 
 {'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
2


In [8]:
# printing the first 10 lines of the data:
for i in range(10):
    print(raw_datasets['train'][i])

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
{'text': 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', 'label': 2}
{'text': "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.", 'label': 2}
{'text': 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.', 'lab

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
# Convert the training data into list of strings
train_texts = [example['text'] for example in raw_datasets['train']]
train_labels = [example['label'] for example in raw_datasets['train']]

In [13]:
print(train_texts[0])
print(train_labels[0])

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
2


In [14]:
# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

In [15]:
# Learn a vocabulary dictionary of all tokens in the raw documents and return term-document matrix
X_train = vectorizer.fit_transform(train_texts)


In [17]:
type(X_train)
print("Shape of X_train: ", X_train.shape)

(120000, 65006)


In [19]:
import numpy as np
# Convert labels to numpy array
y_train = np.array(train_labels)

In [22]:
print("Shape of y_train: ", y_train.shape)

Shape of y_train:  (120000,)


Now lets try the Naive Bayes classifier with a small sample of data:

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np

# Load the dataset
raw_datasets = load_dataset("ag_news")

# Select the first 1000 samples
train_subset = raw_datasets['train'].select(range(1000))

# Convert the train data into list of strings
train_texts = train_subset['text']
train_labels = train_subset['label']

# Initialize a CountVectorizer
vectorizer = CountVectorizer()

# Transform the train data into the BoW representation
X_train = vectorizer.fit_transform(train_texts)

# Convert labels to numpy array
y_train = np.array(train_labels)

# Initialize the Multinomial Naive Bayes classifier
naive_bayes_clf = MultinomialNB()

# Train the classifier
naive_bayes_clf.fit(X_train, y_train)

# Select the first 1000 samples from test set
test_subset = raw_datasets['test'].select(range(1000))

# Convert the test data into list of strings
test_texts = test_subset['text']
test_labels = test_subset['label']

# Transform the test data into the BoW representation
X_test = vectorizer.transform(test_texts)

# Convert labels to numpy array
y_test = np.array(test_labels)

# Predict the labels of the test data
y_pred = naive_bayes_clf.predict(X_test)

# Print the accuracy of the classifier
accuracy = (y_pred == y_test).mean()
print(f"Accuracy: {accuracy}")


Found cached dataset ag_news (/Users/mahnaz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
100%|██████████| 2/2 [00:00<00:00, 376.29it/s]

Accuracy: 0.726
